<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/data_connectors/simple_directory_reader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Directory Reader over a Remote FileSystem

The `SimpleDirectoryReader` is the most commonly used data connector that _just works_.  
By default, it can be used to parse a variety of file-types on your local filesystem into a list of `Document` objects.
Additionaly, it can also be configured to read from a remote filesystem just as easily! This is made possible through the [`fsspec`](https://filesystem-spec.readthedocs.io/en/latest/index.html) protocol.

This notebook will take you through an example of using `SimpleDirectoryReader` to load documents from an S3 bucket. You can either run this against an actual S3 bucket, or a locally emulated S3 bucket via [LocalStack](https://www.localstack.cloud/).

### Get Started

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [2]:
!pip install llama-index s3fs boto3

  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.10.0-py3-none-any.whl (179 kB)
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/12.6 MB 9.5 MB/s eta 0:00:02
   --------------- ------------------------ 5.0/12.6 MB 15.9 MB/s eta 0:00:01
   ---------------- ----------------------- 5.2/12.6 MB 11.0 MB/s eta 0:00:01
   ------------------- -------------------- 6.0/12.6 MB 8.6 MB/s eta 0:00:01
   --------------------- ------------------ 6.8/12.6 MB 7.1 MB/s eta 0:00:01
   ------------------------ --------------- 7.6/12.6 MB 6.5 MB/s eta 0:00:01
   ----------------------------- ---------- 9.2/12.6 MB 6.5 MB/s eta 0:00:01
   ------------------------------ --------- 9.7/12.6 MB 6.2 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 6.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.9.0
    Uninstalling fsspec-2024.9.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.1.0 requires dill<0.3.9,>=0.3.0, but you have dill 0.3.9 which is incompatible.
datasets 3.1.0 requires fsspec[http]<=2024.9.0,>=2023.1.0, but you have fsspec 2024.10.0 which is incompatible.
datasets 3.1.0 requires multiprocess<0.70.17, but you have multiprocess 0.70.17 which is incompatible.
llama-index-embeddings-huggingface 0.4.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which is incompatible.
llama-index-readers-database 0.3.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which is incompatible.
llama-index-readers-deeplake 0.3.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which is incompatible.
open-webui 0.3.35 requires chromadb==0.5.9, but you have chromadb 0.5.20 which is incompatible.


Download Data

In [ ]:
# Use this in Windows to download the file again
!wsl mkdir -p 'data/paul_graham/'
!wsl wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay1.txt'
!wsl wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay2.txt'
!wsl wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay3.txt'


In [3]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay1.txt'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay2.txt'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay3.txt'

The syntax of the command is incorrect.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
!docker run -d -p 4566:4566 -p 4571:4571 localstack/localstack


9c81df6d8c3cd726535f4e6afb1af812d2a92a4a7c9457a6973c6e36909d5c3e


Unable to find image 'localstack/localstack:latest' locally
latest: Pulling from localstack/localstack
2d429b9e73a6: Already exists
14dbff54af92: Pulling fs layer
71ba669986f7: Pulling fs layer
173289c0cbe5: Pulling fs layer
7d12a41bb0ce: Pulling fs layer
aab3a1b85e1e: Pulling fs layer
360ea7496b73: Pulling fs layer
69245e253894: Pulling fs layer
4f4fb700ef54: Pulling fs layer
dfe60959c8e1: Pulling fs layer
8a4fc378c077: Pulling fs layer
b154fb92cb62: Pulling fs layer
e542229f4384: Pulling fs layer
219c42947f0a: Pulling fs layer
6ddfbfdda1f5: Pulling fs layer
6357c482b142: Pulling fs layer
7d12a41bb0ce: Waiting
9c4bf883dd57: Pulling fs layer
aab3a1b85e1e: Waiting
360ea7496b73: Waiting
b4a5d5b8ab34: Pulling fs layer
69245e253894: Waiting
f546920425c7: Pulling fs layer
4f4fb700ef54: Waiting
c9055f1ca157: Pulling fs layer
dfe60959c8e1: Waiting
8a4fc378c077: Waiting
2e72f908c1de: Pulling fs layer
b154fb92cb62: Waiting
e96e7155d293: Pulling fs layer
219c42947f0a: Waiting
ec5d1383c616: Pulli

In [8]:
# create a test-bucket in S3
import boto3

endpoint_url = (
    "http://localhost:4566"  # use this line if you are using S3 via localstack
)
# endpoint_url = None  # use this line if you are using real AWS S3
bucket_name = "llama-index-test-bucket"
#s3 = boto3.resource("s3", endpoint_url=endpoint_url)
# Set up boto3 resource with dummy credentials
s3 = boto3.resource(
    "s3",
    endpoint_url=endpoint_url,
    aws_access_key_id="dummy_access_key",
    aws_secret_access_key="dummy_secret_key",
)
s3.create_bucket(Bucket=bucket_name)
bucket = s3.Bucket(bucket_name)
# put the paul graham essays in the test-bucket in various subdirectories
bucket.upload_file(
    "data/paul_graham/paul_graham_essay1.txt", "essays/paul_graham_essay1.txt"
)
bucket.upload_file(
    "data/paul_graham/paul_graham_essay2.txt",
    "essays/more_essays/paul_graham_essay2.txt",
)
bucket.upload_file(
    "data/paul_graham/paul_graham_essay3.txt",
    "essays/even_more_essays/paul_graham_essay3.txt",
)

In [9]:
from llama_index.core import SimpleDirectoryReader

In [ ]:
# create the filesystem using s3fs
from s3fs import S3FileSystem

s3_fs = S3FileSystem(anon=False,
                    key="dummy_access_key",  # Dummy access key for LocalStack
                    secret="dummy_secret_key",  # Dummy secret key for LocalStack
                    endpoint_url=endpoint_url)

Load specific files 

In [11]:
reader = SimpleDirectoryReader(
    input_dir=bucket_name,
    fs=s3_fs,
    recursive=True,  # recursively searches all subdirectories
)

NoCredentialsError: Unable to locate credentials

In [ ]:
docs = reader.load_data()
print(f"Loaded {len(docs)} docs")

Loaded 3 docs


Load all (top-level) files from directory

In [ ]:
reader = SimpleDirectoryReader(input_dir="./data/paul_graham/")

In [ ]:
docs = reader.load_data()
print(f"Loaded {len(docs)} docs")

# show the metadata of each document
for idx, doc in enumerate(docs):
    print(f"{idx} - {doc.metadata}")

Loaded 3 docs
0 - {'file_path': '/Users/sourabhdesai/workspace/llama_index/docs/docs/examples/data_connectors/data/paul_graham/paul_graham_essay1.txt', 'file_name': 'paul_graham_essay1.txt', 'file_type': 'text/plain', 'file_size': 0, 'creation_date': '2024-03-21', 'last_modified_date': '2024-03-21'}
1 - {'file_path': '/Users/sourabhdesai/workspace/llama_index/docs/docs/examples/data_connectors/data/paul_graham/paul_graham_essay2.txt', 'file_name': 'paul_graham_essay2.txt', 'file_type': 'text/plain', 'file_size': 0, 'creation_date': '2024-03-21', 'last_modified_date': '2024-03-21'}
2 - {'file_path': '/Users/sourabhdesai/workspace/llama_index/docs/docs/examples/data_connectors/data/paul_graham/paul_graham_essay3.txt', 'file_name': 'paul_graham_essay3.txt', 'file_type': 'text/plain', 'file_size': 0, 'creation_date': '2024-03-21', 'last_modified_date': '2024-03-21'}


In [ ]:
docs = reader.load_data()
print(f"Loaded {len(docs)} docs")

# show the metadata of each document
for idx, doc in enumerate(docs):
    print(f"{idx} - {doc.metadata}")

Loaded 3 docs
0 - {'file_path': '/Users/sourabhdesai/workspace/llama_index/docs/docs/examples/data_connectors/data/paul_graham/paul_graham_essay1.txt', 'file_name': 'paul_graham_essay1.txt', 'file_type': 'text/plain', 'file_size': 0, 'creation_date': '2024-03-21', 'last_modified_date': '2024-03-21'}
1 - {'file_path': '/Users/sourabhdesai/workspace/llama_index/docs/docs/examples/data_connectors/data/paul_graham/paul_graham_essay2.txt', 'file_name': 'paul_graham_essay2.txt', 'file_type': 'text/plain', 'file_size': 0, 'creation_date': '2024-03-21', 'last_modified_date': '2024-03-21'}
2 - {'file_path': '/Users/sourabhdesai/workspace/llama_index/docs/docs/examples/data_connectors/data/paul_graham/paul_graham_essay3.txt', 'file_name': 'paul_graham_essay3.txt', 'file_type': 'text/plain', 'file_size': 0, 'creation_date': '2024-03-21', 'last_modified_date': '2024-03-21'}


Create an iterator to load files and process them as they load

In [ ]:
reader = SimpleDirectoryReader(
    input_dir=bucket_name,
    fs=s3_fs,
    recursive=True,
)

all_docs = []
for docs in reader.iter_data():
    for doc in docs:
        # do something with the doc
        doc.text = doc.text.upper()
        all_docs.append(doc)

print(len(all_docs))

3


Exclude specific patterns on the remote FS

In [ ]:
reader = SimpleDirectoryReader(
    input_dir=bucket_name,
    fs=s3_fs,
    recursive=True,
    exclude=["essays/more_essays/*"],
)

all_docs = []
for docs in reader.iter_data():
    for doc in docs:
        # do something with the doc
        doc.text = doc.text.upper()
        all_docs.append(doc)

print(len(all_docs))
all_docs

2


[Document(id_='8329e3e6-a16e-4aca-8a85-3b338d888925', embedding=None, metadata={'file_path': 'llama-index-test-bucket/essays/even_more_essays/paul_graham_essay3.txt', 'file_name': 'paul_graham_essay3.txt', 'file_type': 'text/plain', 'file_size': 0}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='cc681e3c-47f6-4e34-8e8b-e2bb0b44ea2d', embedding=None, metadata={'file_path': 'llama-index-test-bucket/essays/paul_graham_essay1.txt', 'file_name': 'paul_graham_essay1.txt', 'file_type': 'text/plain', 'file_size': 0}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date

Async execution is available through `aload_data`

In [ ]:
import nest_asyncio

nest_asyncio.apply()

reader = SimpleDirectoryReader(
    input_dir=bucket_name,
    fs=s3_fs,
    recursive=True,
)

all_docs = await reader.aload_data()

print(len(all_docs))

3
